In [ ]:
from pyzm.interface import ZMESConfig
from pyzm.interface import GlobalConfig, MLAPI_DEFAULT_CONFIG as DEFAULT_CONFIG
from pyzm.helpers.pyzm_utils import LogBuffer
from pyzm.api import ZMApi
import yaml

yaml_file_path = './yaml'
CONFIG_PATH = rf'{yaml_file_path}/objectconfig.yml'
SECRETS_LOCAL = rf'{yaml_file_path}/secrets.yml'

# Load local secrets from yaml file
def LoadSecretsYAML(secrets_file: str) -> dict:
    with open(secrets_file, 'r') as file:
        secrets_yml = yaml.safe_load(file)
    return (secrets_yml)

global g
g = GlobalConfig()
g.logger = LogBuffer()

mlc: ZMESConfig = ZMESConfig(CONFIG_PATH, DEFAULT_CONFIG, "mlapi") 
g.config = mlc.config

# Set credentials
secret = LoadSecretsYAML(SECRETS_LOCAL)
api_options = {
    "apiurl": secret['secret']['ZM_API_PORTAL'],
    "portalurl": secret['secret']['ZM_PORTAL'],
    'user': secret['secret']['ZM_USER'],
    'password': secret['secret']['ZM_PASSWORD'],
    'basic_auth_user': secret['secret']['ZM_BASIC_USER'],
    'basic_auth_password': secret['secret']['ZM_BASIC_PASS']
}

# Connect to ZM-API
g.api = ZMApi(options=api_options)

# Check if connection is established
version = g.api.version()
print(f"ZM-Version: {version['zm_version']} - Status: {version['status']}")

#### Query for zones for a specific monitor

In [ ]:
from  pprint import pprint
list_of_zones = g.api.zones()
list_of_zones = sorted(list_of_zones, key=lambda x: x.zone['Zone']['MonitorId'])
for zone in list_of_zones:
    print(f"{zone.zone['Zone']['MonitorId']:4} - {zone.zone['Zone']['Name']:20} - {zone.zone['Zone']['Type']}")


In [ ]:
import re

# Get Zone-Coordinates for Monitor ID No 9
zones_of_mid = 9

def getZoneCoordinates(MonID: int) -> list:
    mon_zone_coords = []

    # Get all zones
    list_of_zones = g.api.zones()

    for zone in list_of_zones:
        if (     (zone.zone['Zone']['MonitorId'] == MonID)
             and (zone.zone['Zone']['Type'] == 'Active')
             and (re.findall("event", zone.zone['Zone']['Name']))):

            mon_zone_coords.append(zone.zone['Zone']['Coords'])

    return mon_zone_coords

ZMZoneCoordinates = getZoneCoordinates(zones_of_mid)

# Format [[1595, 922], [996, 917]]]
for zones in ZMZoneCoordinates:
    zones_pts = zones.split(' ')
    zone_coordinates = [list(map(int, x.split(','))) for x in zones_pts]

print(f"Monitor: {zones_of_mid} - Zone-Coordinates: {zone_coordinates}")

### Get zone from Inkscape image

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import re
from image_event_folder import *

home_dir = os.path.expanduser("~")
svg_zone_coord = LoadZones(f"{home_dir}/Development/zm-programming/zone_object_location/Inkscape/zone_mon_8.html")

start_of_polygon_coord = svg_zone_coord.split('moveTo')
polygon_coord_raw = re.findall(r"[-+]?\d*\.\d+|\d+", start_of_polygon_coord[1])

# Convert list of strings to list of integers
polygon_coord = [int(float(x)) for x in polygon_coord_raw]
#print(polygon_coord)

# Convert list of tuple
polygon_coord_tupel = ([(polygon_coord[i], polygon_coord[i+1]) for i in range(0, len(polygon_coord), 2)])
#print(polygon_coord_tupel)

# Convert list of lists - Format eg: [[1595, 922], [996, 917]]]
polygon_coord_list = [list(i) for i in polygon_coord_tupel]
print(polygon_coord_list)
zone_coordinates_Inkscape = polygon_coord_list

# Create a blank image and draw the polygon
height, width, c = 1080, 1920, 3
gray_level = 127
blank_image = gray_level * np.ones(shape=(height, width, c), dtype=np.uint8)

def draw_polygon(image, ptr: list):
    pts = np.array(ptr, np.int32)    
    cv2.polylines(image, [pts], True, (255,0,0), thickness=12)

draw_polygon(blank_image, zone_coordinates_Inkscape)

zone_coordinates = zone_coordinates_Inkscape

plt.imshow(blank_image)

#### Draw zones into blank picture

In [ ]:
import numpy as np
import cv2
import imutils
import matplotlib.pyplot as plt
from pathlib import Path

# get size of image of a specific monitor
home = str(Path.home())
image = cv2.imread(home + '/Development/zm-programming/objdet_pynb/pictures/zones_object_detection/00072-capture.jpg')

height, width, c = image.shape
gray_level = 127
blank_image = gray_level * np.ones(shape=(height, width, c), dtype=np.uint8)

### Rectangle definition
def draw_rectangle(image, x, y, width, height):
    cv2.rectangle(image, (x, y), (x + width, y + height), (255,0,0), thickness=12)

def draw_rectangle_color(image, x, y, width, height, color=(255,0,0)):
    cv2.rectangle(image, (x, y), (x + width, y + height), color, thickness=12)

def draw_polygon(image, ptr: list):
    pts = np.array(ptr, np.int32)    
    cv2.polylines(image, [pts], True, (255,0,0), thickness=12)

### Rectangle definition end
print(zone_coordinates)
draw_polygon(blank_image, zone_coordinates)

plt.imshow(blank_image)


#### Assuming there is a yolo finding

In [ ]:
draw_rectangle_color (blank_image, 500, 200, 200, 400 , (0, 120, 0))


plt.imshow(blank_image)

In [ ]:
# convert  the image to a grayscale type
gray_scaled = cv2.cvtColor(blank_image, cv2.COLOR_BGR2GRAY)

# Thresholding the image
thresh = cv2.threshold(gray_scaled, 125, 125, cv2.THRESH_BINARY_INV)[1]

# detecting and drawing contours
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)

# get the size of the zone
size_of_zone = cv2.contourArea(contours[0])

output = blank_image.copy()

# loop over each contour found
for contour in contours:
    # outline and display them, one by one.
    cv2.drawContours(output, [contour], -1, (240, 0, 159), 13) 

plt.imshow(output)

print (f"There are {len(contours)} objects here.")
if len(contours) == 1:
    print ("Person in Zone detected")
else:
    print ("No Person in Zone detectd")

### Assuming there is another yolo finding

In [ ]:
blank_image = np.full((height, width, 3), (128,128,128), np.uint8)
draw_polygon(blank_image, zone_coordinates)

draw_rectangle_color (blank_image, 1100, 400, 200, 400 , (0, 120, 0))

plt.imshow(blank_image)

In [ ]:
#draw_rectangle (blank_image,  400,  400,  200,  400)

# convert to a grayscale
gray_scaled = cv2.cvtColor(blank_image, cv2.COLOR_BGR2GRAY)

# Thresholding the image
thresh = cv2.threshold(gray_scaled, 125, 125, cv2.THRESH_BINARY_INV)[1]

# Detecting and drawing contours
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
output = blank_image.copy()

# loop over each contour found
for contour in contours:
    cv2.drawContours(output, [contour], -1, (240, 0, 159), 13) 

plt.imshow(output)

print (f"There are {len(contours)} objects here.")
if len(contours) == 1:
    print ("Person in Zone detected")
else:
    print ("No Person in Zone detectd")

### Calc event

In [ ]:
###################
#  Chose EventID  
###################
EventID = 31272

# API call to get all zones
Event, Monitor, Frame = g.api.get_all_event_data(EventID)
event_folder = Event["FileSystemPath"]

# open the folder in file manager
!caja $event_folder

In [ ]:
import numpy as np
import cv2
import imutils
import matplotlib.pyplot as plt
from pathlib import Path
from image_event_folder import *

###################
#  Chose EventID  
###################
# EventID = 31272

EventsFolder = '/mnt/DiskVerbatimVi55/ZMEvents'

# take size from riginal image
Event, Monitor, Frame = g.api.get_all_event_data(EventID)
mid = Monitor['Id']

event_folder = getEventFolder(EventsFolder, EventID)

json_data = LoadObjectsJson(event_folder)
image_obj = cv2.imread(event_folder + '/' + 'objdetect.jpg')
height, width, c = image_obj.shape

# create blank_image
gray_level = 127
blank_image = gray_level * np.ones(shape=(height, width, c), dtype=np.uint8)

labels = json_data['labels']
boxes = json_data['boxes']
object_boxes_list=[]
for counter, value in enumerate(labels, start=0):
    print(counter, value)
    #if value == 'person':
    if value in g.config['object_detection_pattern']:
        object_boxes_list.append(boxes[counter])

print(f"Object boxes: {object_boxes_list}")


# Get Zone-Coordinates for Monitor ID
zone_coordinates = getZoneCoordinates(mid)
print (f"zone_coordinates: {zone_coordinates}")

for zones in zone_coordinates:
    zones_pts = zones.split(' ')
    zones_pts = [list(map(int, x.split(','))) for x in zones_pts]
    draw_polygon(blank_image, zones_pts)

print(f"Monitor: {mid} - Zone-Coordinates: {zone_coordinates}")


In [ ]:
import numpy as np
import cv2
import imutils
import matplotlib.pyplot as plt
from pathlib import Path
from image_event_folder import *

person_zone_state=list()
image_list = []

for object_box_counter, object_box in enumerate(object_boxes_list):

    blank_image = gray_level * np.ones(shape=(height, width, c), dtype=np.uint8)

    draw_polygon(blank_image, zones_pts)

    # get lenght of the object bounding box
    # convert image to a grayscale
    gray_scaled = cv2.cvtColor(blank_image, cv2.COLOR_BGR2GRAY)

    # thresholding the image
    thresh = cv2.threshold(gray_scaled, 125, 125, cv2.THRESH_BINARY_INV)[1]

    # find contours in thresholded image
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    size_of_zone = cv2.contourArea(contours[0])


    draw_rectangle_color(blank_image, 
                        object_box[0], 
                        object_box[1], 
                        object_box[2]-object_box[0], 
                        object_box[3]-object_box[1], 
                        (0, 120, 0))

    # convert  the image to a grayscale type
    gray_scaled = cv2.cvtColor(blank_image, cv2.COLOR_BGR2GRAY)

    # Thresholding the image
    thresh = cv2.threshold(gray_scaled, 125, 125, cv2.THRESH_BINARY_INV)[1]

    # find the contours in the thresholded image
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)

    size_of_zone_incl_object = cv2.contourArea(contours[0])

    image_out = blank_image.copy()
    for contour in contours:
        # display one by one.
        cv2.drawContours(image_out, [contour], -1, (240, 0, 159), 10) 

    # weighted input image and the overlay
    image_out = result = cv2.addWeighted(image_out, .5, image_obj, .5, 0)

    image_list.append(image_out)

    #print (f"No objects {len(contours)}")
    if len(contours) > len(zone_coordinates):
        person_zone_state.append({object_box_counter : 'Outside'})
    else:
        # if the zone has increased in size
        if size_of_zone_incl_object > size_of_zone:
            person_zone_state.append({object_box_counter: 'Partially inside'})
        else:
            person_zone_state.append({object_box_counter: 'Completely inside'})

    #person_zone_state.append({object_box_counter : 'Outside'})


print (person_zone_state)

### Handle partly in zone

In [ ]:
blank_image = np.full((height, width, 3), (128,128,128), np.uint8)

draw_polygon(blank_image, zone_coordinates)

draw_rectangle_color (blank_image, 900, 400, 200, 400 , (0, 120, 0))

plt.imshow(blank_image)



### Draw zones on images

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

path_object_zones = './pictures/zones_object_detection/objdetect_zones'

# figure size in inches
rcParams['figure.figsize'] = 10,10

cols = 1
rows = len(image_list)

# display images
fig, axes = plt.subplots(rows, cols, figsize=(16,12))

for i, img in enumerate(image_list): 
    cv2.imwrite( f'{path_object_zones}_{i}.jpg', img)
    axes[i].imshow(img)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
from networkx import center

path_object_zones = './pictures/zones_object_detection/objdetect_zones'

image_paint = image_out.copy()

for number, zone_state in enumerate(person_zone_state):

    radius = 50
    center_x = (number+1) * (radius*2) + ((number+1)*10)

    # Center coordinates 
    center_coordinates = (center_x, 150)

    print(f"zone_state: {zone_state}")
    print(f"zone_state: {list(zone_state.values())[0]}")

    if list(zone_state.values())[0] == 'Completely inside':
        # Red color in RGB 
        color = (255, 0, 0) 
    elif list(zone_state.values())[0] == 'Partially inside':
        # Red color in RGB 
        color = (220, 125, 125)
    else:
        # Grean color in RGB
        color = (0, 255, 0)

    # Line thickness of -1 px 
    thickness = -1
    
    # Using cv2.circle() method 
    image_paint = cv2.circle(image_paint, center_coordinates, radius, color, thickness) 

    plt.imshow(image_paint)